In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import readdy

In [ ]:
n_particles = 1000
diff = 1.0
rate = 0.1

name = "nparticles.h5"

data_dir = "/home/chris/workspace/data/workshop"
out_file = os.path.join(data_dir, name)
n_steps = 10000
dt = 1e-2

In [ ]:
system = readdy.ReactionDiffusionSystem(
    box_size=[20.,20.,20.],
    periodic_boundary_conditions=[True, True, True],
    unit_system=None)

In [ ]:
system.add_species("A", diff)
system.add_species("B", diff)

In [ ]:
system.reactions.add("react: A +(1.) A -> B", rate=rate)

In [ ]:
simulation = system.simulation("SingleCPU")

In [ ]:
simulation.output_file = out_file
simulation.record_trajectory()
simulation.observe.number_of_particles(1, ["A","B"], callback=lambda x: print("A {}, B {}".format(x[0],x[1])))

In [ ]:
origin = np.array([-10.,-10.,-10.])
extent = np.array([20.,20.,20.])
init_pos = np.random.uniform(size=(n_particles, 3)) * extent + origin
simulation.add_particles("A", init_pos)

In [ ]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)
simulation.run(n_steps,dt)

In [ ]:
traj = readdy.Trajectory(out_file)

In [ ]:
times, counts = traj.read_observable_number_of_particles()

In [ ]:
counts.shape

In [ ]:
plt.plot(times, counts[:,0], label="A")
plt.plot(times, counts[:,1], label="B")
plt.legend()

In [ ]:
import scipy.optimize as so

def func(t, k):
    return 1./(1./float(n_particles) + k * t)

popt, pcov = so.curve_fit(func, times, counts[:,0])

f = lambda t: func(t, popt)

plt.figure(figsize=(4,3))
plt.plot(times/1000., counts[:,0]/1000., label=r"data $a(t)$")
plt.plot(times/1000., f(times)/1000., label=r"fit $(a_0^{-1}+kt)^{-1}$")
plt.legend()
plt.xlabel("Time/a.u.")
plt.ylabel("Concentration/a.u.")
plt.title("$D="+str(diff)+", \lambda="+str(rate)+"$")
plt.gcf().tight_layout()
plt.savefig("fusion_diff_"+str(diff)+".pdf")
plt.show()